In [1]:
from __future__ import print_function, division
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as im
from PIL import Image
import skimage
from tqdm import tqdm, tqdm_notebook
import re
import math

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torchvision.models as models
from torch.autograd import Variable
from sklearn import linear_model
from torch import nn
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.decomposition import PCA

import pickle

%load_ext autoreload
%autoreload 2

In [2]:
data_dir = 'dataset/broden1_227/'
df = pd.read_csv(data_dir + 'processed_index.csv')

In [3]:
def filter_by_category(df, category, train):
    pattern = "[%i]" % category
    split = 'train' if train else 'val'
    return df[df['features'].str.contains(pattern, regex=False) & df['split'].str.match(split)].index
    
def filter_by_not_category(df, category, train):
    pattern = "[%i]" % category
    split = 'train' if train else 'val'
    return df[~df['features'].str.contains(pattern, regex=False) & df['split'].str.match(split)].index


In [4]:
device = torch.device('cuda:0')

In [5]:
model = models.resnet50(pretrained=True)

model.to(device)
print("model initialized")

model initialized


In [6]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [14]:
outputs = []
    
def layer_hook(module, input, output):
    layer_output = output.data.cpu()
    m = nn.AvgPool2d(4)
    layer_output = m(layer_output)
    outputs.append(layer_output)
# handle = model.layer1[2].bn3.register_forward_hook(layer_hook)
# handle = model.layer2[3].bn3.register_forward_hook(layer_hook)
# handle = model.layer3[5].bn3.register_forward_hook(layer_hook)
handle = model.layer4[2].bn3.register_forward_hook(layer_hook)

In [15]:
class CategoryDataSet(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.dataframe = df
        self.root_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        imgfile = os.path.join(self.root_dir,
                                self.dataframe.at[idx, 'image'])
        image = Image.open(imgfile)
        
        if self.transform:
            image = self.transform(image)
            
        sample = (image, idx)
        
        return sample

In [16]:
tf = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])

dataset = CategoryDataSet(df, data_dir + 'images/', transform=tf)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=25, shuffle=False, num_workers=1)

In [17]:
torch.cuda.empty_cache()

In [18]:
TRAIN_MAX_INDEX = 44403

In [19]:
predictions = []
# run dataset through model
for inputs, idx in tqdm(dataloader):
    inputs = inputs.to(device)
    model(inputs)

100%|██████████| 2533/2533 [04:09<00:00, 10.16it/s]


In [20]:
def transform_outputs(outputs):
    stacked_outputs = torch.cat(outputs, dim=0)
#     m = nn.AvgPool2d(4)
#     max_pool_outputs = m(stacked_outputs)
    max_pool_outputs = stacked_outputs
    flattened_outputs = max_pool_outputs.reshape((max_pool_outputs.shape[0], np.prod(max_pool_outputs.shape[1:])))
    normalized_outputs = normalize(flattened_outputs)
    return normalized_outputs

In [21]:
layer = 'layer4'

In [22]:
# with open("outputs/%s.pickle" % layer, 'wb') as handle:
#     pickle.dump(outputs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
stacked_outputs = np.concatenate(outputs)
max_pool_outputs = stacked_outputs
flattened_outputs = max_pool_outputs.reshape((max_pool_outputs.shape[0], np.prod(max_pool_outputs.shape[1:])))
normalized_outputs = normalize(flattened_outputs)

In [24]:
normalized_outputs.shape

(63305, 2048)

In [25]:
# transform all activations
# transformed_outputs = transform_outputs(outputs)
# transformed_outputs = transform_outputs(outputs['layer4'])
pca = PCA(n_components=100)
pca.fit(normalized_outputs[:TRAIN_MAX_INDEX]) # just do it on training data

with open("pca/%s.pickle" % layer, 'wb') as handle:
    pickle.dump(pca, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [26]:
transformed_outputs = pca.transform(normalized_outputs)
outputs = transformed_outputs

In [27]:
outputs.shape

(63305, 100)

In [28]:
SGD_MAX_ITERATIONS = 400
SGD_LOSS = "hinge"
SGD_PENALTY = "l2"

import random

def train_clf_for_category(df, category, activations):
    # Filter out category data from dataframe
    pos_train_indices = filter_by_category(df, category, True)
    negative_train_indices = filter_by_not_category(df,category, True)
    pos_val_indices = filter_by_category(df, category, False)
    negative_val_indices = filter_by_not_category(df,category, False)        
    
    if(len(pos_train_indices) == 0):
        print("Empty set found for category %i" % category)
        return(None, 0)
    
    # generate balacnced train set
    pos_train = [activations[i] for i in pos_train_indices]
    negative_train = [activations[i] for i in random.sample(negative_train_indices, min(len(pos_train_indices), len(negative_train_indices)))]
    pos_val = [activations[i] for i in pos_val_indices]
    negative_val = [activations[i] for i in random.sample(negative_val_indices, min(len(pos_val_indices), len(negative_val_indices)))]
    
    # generate training and validation data for SGDClassifier
    train = pos_train + negative_train
    val = pos_val + negative_val
    
    train_labels = np.append(np.ones(len(pos_train)), np.zeros(len(negative_train)))
    val_labels = np.append(np.ones(len(pos_val)), np.zeros(len(negative_val)))
    clf = linear_model.SGDClassifier(max_iter=min(np.ceil(10**6 / len(train_labels)), SGD_MAX_ITERATIONS), loss=SGD_LOSS, early_stopping=True, penalty=SGD_PENALTY, average=True, eta0=1.5)
    clf.fit(train, train_labels)
    return (clf, clf.score(val, val_labels))


In [29]:
clf, score = train_clf_for_category(df, 1, outputs)
print(score)

/home/conner/anaconda3/envs/torch2/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.9791016348341358


In [30]:
# chosen based on frequency in the dataset
MAX_CLASSES = 150
SCORE_THRESH = 0.75
sgd_classifiers = {}

In [31]:
for i in range(1, MAX_CLASSES):
    clf, score = train_clf_for_category(df, i, outputs)
    if(score > SCORE_THRESH):
        sgd_classifiers[i] = (clf, score)
    else:
        print("ignoring clf for category %i with score %f" % (i, score))

ignoring clf for category 12 with score 0.652450
ignoring clf for category 14 with score 0.703454
ignoring clf for category 15 with score 0.722811
ignoring clf for category 17 with score 0.721992
ignoring clf for category 20 with score 0.709237
ignoring clf for category 22 with score 0.692779
ignoring clf for category 23 with score 0.707806
ignoring clf for category 24 with score 0.720226
ignoring clf for category 27 with score 0.715784
ignoring clf for category 28 with score 0.707865
ignoring clf for category 30 with score 0.718845
ignoring clf for category 34 with score 0.741709
ignoring clf for category 35 with score 0.729223
ignoring clf for category 36 with score 0.701427
ignoring clf for category 41 with score 0.744082
ignoring clf for category 45 with score 0.737057
ignoring clf for category 46 with score 0.714679
ignoring clf for category 48 with score 0.721154
ignoring clf for category 49 with score 0.717349
Empty set found for category 59
ignoring clf for category 59 with sco

In [36]:
scores = []
multiplied_accuracy = 1
for key, value in sgd_classifiers.items():
    scores.append(value[1])
    multiplied_accuracy = multiplied_accuracy * value[1]

print (reduce(lambda x, y: x + y, scores) / len(scores))

0.8283000747152544


In [33]:
sgd_classifiers.get(1)

(SGDClassifier(alpha=0.0001, average=True, class_weight=None,
        early_stopping=True, epsilon=0.1, eta0=1.5, fit_intercept=True,
        l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=23.0,
        n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
        power_t=0.5, random_state=None, shuffle=True, tol=None,
        validation_fraction=0.1, verbose=0, warm_start=False),
 0.9791016348341358)

In [34]:
import pickle

# Store data (serialize)
with open('classifiers-%s.pickle' % layer, 'wb') as handle:
    pickle.dump(sgd_classifiers, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [37]:
len (sgd_classifiers)

90